In [ ]:
import tf import pandas as pd from keras.src.layers import Dropout from keras import layers from tensorflow.keras.optimizers import legacy from sklearn.preprocessing import MinMaxScaler from keras.models import Sequential import numpy as np import matplotlib.pyplot as plt from tensorflow.keras.models import Sequential from tensorflow.keras.layers import LSTM from tensorflow.keras.layers import Dense, Dropout from matplotlib import pyplot as plt from sklearn.preprocessing import StandardScaler import seaborn as sns from keras.src.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard from sklearn.metrics import mean_squared_error as mse from sklearn.model_selection import train_test_split from keras.src.losses import MeanSquaredError, mean_absolute_error, mean_squared_error from keras.src.metrics import RootMeanSquaredError from keras.src.engine.input_layer import InputLayer import math import keras from keras.layers import * from sklearn.preprocessing import MinMaxScaler from sklearn.metrics import mean_squared_error from sklearn.metrics import mean_absolute_error from keras.callbacks import EarlyStopping

In [ ]:
df = pd.read_csv('../Datasets/Datasets_Reduced/dataset_historic_zone.csv') df['nPeople'] = df['C1'] + df['C2'] cols=['C1','C2','C3','C4','D1','Year','Tourist_Local','Holiday','Time','Day Moment','Hour','Parish','City Zone'] df = df.drop(columns=cols) df['Datetime'] = pd.to_datetime(df['Datetime']) df = df.sort_values(by=['Grid_ID', 'Datetime']) df.index = pd.to_datetime(df['Datetime'], format='%d.%m.%Y %H:%M:%S') 

In [ ]:
df.info()

In [ ]:
nPeople = df['nPeople'] nPeople.plot()

In [ ]:
df.head()

In [14]:
  from keras.src.regularizers import l2 from keras.src.optimizers import Adam  data = df.copy() data = data.groupby('Grid_ID') grid_count = 0  for grid_name, grid_data in data:     if grid_count >= 2:         break      cols=['Grid_ID','Datetime']     grid_data=grid_data.drop(columns=cols)     grid_data = grid_data.reindex(columns=['nPeople', 'Weekday', 'Day','Month'])     grid_data = grid_data.astype(float)      print(grid_data.info())      test_split=round(len(grid_data)*0.20)      df_for_training=grid_data[:-4017]     df_for_testing=grid_data[-4017:]        scaler = StandardScaler()     df_for_training_scaled = scaler.fit_transform(df_for_training)     df_for_testing_scaled=scaler.transform(df_for_testing)      #[[ [nPeople1, Month1, Day1, Weekday1], [nPeople2, Month2, Day2, Weekday2],[nPeople3, Month3, Day3, Weekday3]]] [nPeople4, Month4, Day4, Weekday4]     #[[ [nPeople2, Month2, Day2, Weekday2],[nPeople3, Month3, Day3, Weekday3]]] [nPeople4, Month4, Day4, Weekday4] [nPeople5, Month5, Day5, Weekday5]     #[[ [nPeople3, Month3, Day3, Weekday3]]] [nPeople4, Month4, Day4, Weekday4],[nPeople5, Month5, Day5, Weekday5]]] [nPeople6, Month6, Day6, Weekday6]     def createXY(dataset,n_past):         dataX = []         dataY = []         for i in range(n_past, len(dataset)):             dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])             dataY.append(dataset[i,0])         return np.array(dataX),np.array(dataY)      trainX,trainY=createXY(df_for_training_scaled,96*5)     testX,testY=createXY(df_for_testing_scaled,96*5)      print("trainX Shape-- ",trainX.shape)     print("trainY Shape-- ",trainY.shape)      print("testX Shape-- ",testX.shape)     print("testY Shape-- ",testY.shape)      look_back=96*5     cell_shape=4     model = Sequential()     model.add(InputLayer((96*5, 4)))     model.add(LSTM(64, activation='sigmoid', return_sequences=True))     model.add(LSTM(32, activation='sigmoid', return_sequences=False))     model.add(Dense (50))     model.add(Dropout(0.2))     model.add(Dense(1))           model_checkpoint = ModelCheckpoint(f'model_{grid_name}/', save_best_only=True)     model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=['mse', 'mae'])      history = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=32, epochs=10,callbacks=[model_checkpoint])      plt.plot(history.history['loss'], label='Training loss')     plt.plot(history.history['val_loss'], label='Validation loss')     plt.legend()     plt.show()      prediction=model.predict(testX)     print("prediction
 ", prediction)     print("
 Prediction Shape-",prediction.shape)      prediction_copies_array = np.repeat(prediction,4, axis=-1)     pred=scaler.inverse_transform(np.reshape(prediction_copies_array,(len(prediction),4)))[:,0]      original_copies_array = np.repeat(testY,4, axis=-1)     original_copies_array.shape     original=scaler.inverse_transform(np.reshape(original_copies_array,(len(testY),4)))[:,0]      print('=============================
 ')     print("Pred Values-- " ,pred)     print("
 Original Values-- ",original)     print('=============================
 ')      plt.plot(original, color = 'red', label = 'Real  Number of People')     plt.plot(pred, color = 'blue', label = 'Predicted  Number of People')     plt.title(' Number of People Prediction')     plt.xlabel('Timestamp')     plt.ylabel(' Number of People')     plt.legend()     plt.show()      grid_count += 1

KeyboardInterrupt: 